# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [12]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [2]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [8]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [3]:
# Get gpt-4o-mini to answer, with streaming
system_prompt = "You are an expert in python and your job is to help users with the questions related to python code. For any query that the user is posting, provide an indepth explanation of the answer with 3-4 examples and best practices."



In [4]:
print(system_prompt)

You are an expert in python and your job is to help users with the questions related to python code. For any query that the user is posting, provide an indepth explanation of the answer with 3-4 examples and best practices.


In [5]:
def get_user_prompt(query):
    user_prompt = f"Please explain the below code.\n {query}"
    return user_prompt

In [19]:
def get_query_response(query, stream=False):
    response = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(query)}
        ],
        stream=stream
    )

    if stream:
         streaming = ""
         display_handle = display(Markdown(""), display_id=True)
         for chunk in response:
            streaming += chunk.choices[0].delta.content or ''
            streaming = streaming.replace("```","").replace("markdown", "")
            update_display(Markdown(streaming), display_id=display_handle.display_id)
    else:
        result =  response.choices[0].message.content
        display(Markdown(result))


In [20]:
get_query_response(question,True)

The line of code you provided utilizes several key Python concepts, including generator expressions, the `yield` statement, and set comprehensions. Let's break it down step by step to understand its functionality and purpose.

### Code Explanation

**The Code:**
python
yield from {book.get("author") for book in books if book.get("author")}


#### Components Explained:

1. **`yield from`**:
   - The `yield` keyword is used in Python to turn a function into a generator. A generator function can yield multiple values at different points in time, instead of returning a single value.
   - The `yield from` expression allows you to yield all values from an iterable (like a list, tuple, or set) as if they were yielded from the generator itself.
   - This is useful for delegating part of your generator’s operation to another generator or iterable.

2. **Set Comprehension**:
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - It iterates over `books`, which should be a list (or iterable) of dictionaries.
   - For each `book` in `books`, it retrieves the value associated with the key `"author"` using `book.get("author")`.
   - The `if book.get("author")` part ensures that only books with a non-empty or non-None author are included in the resulting set. This helps to filter out any entries that do not have a valid author.

3. **Overall Functionality**:
   - The complete line creates a set of unique author names from a collection of book records. By using a set, it ensures that each author appears only once, even if multiple books are authored by the same person.
   - Finally, the `yield from` keyword allows for yielding these unique author names one at a time from a generator.

### Why Use This Code?

1. **Efficiency**: By using a generator and `yield from`, you can generate authors one at a time, which can be more memory-efficient than creating a complete list of authors and keeping it in memory.
  
2. **Uniqueness**: Using a set comprehension inherently ensures that all authors are unique. If the same author appears multiple times in the `books`, they will only show once in the final output.

3. **Readable**: The use of generator and comprehensions makes the code concise. It's a Pythonic way to perform operations on collections.

### Practical Examples

Let's illustrate this with a few practical examples.

#### Example 1: Basic Usage
python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 1"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 2"},
]

def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

authors = list(get_authors(books))
print(authors)  # Output: ['Author 1', 'Author 2']


#### Example 2: Avoiding Duplicates
python
books = [
    {"title": "Book A", "author": "Author X"},
    {"title": "Book B", "author": "Author X"},
    {"title": "Book C", "author": "Author Y"},
]

authors = list(get_authors(books))
print(authors)  # Output: ['Author X', 'Author Y']


#### Example 3: Handling Edge Cases
python
books = [
    {"title": "Book A", "author": ""},
    {"title": "Book B", "author": None},
    {"title": "Book C", "author": "Author Z"},
]

authors = list(get_authors(books))
print(authors)  # Output: ['Author Z']


### Best Practices
1. **Always Use `get`**: When accessing dictionary keys that might not exist, use the `get()` method to avoid `KeyError`.
  
2. **Filter Early**: Filter out empty or irrelevant data points early in your comprehension to keep your data clean.

3. **Consider Performance**: Using `yield from` helps you manage memory more efficiently, especially with large datasets.

4. **Documentation**: Always provide documentation for your functions to clarify what your generators yield, particularly when using complex comprehensions.

By following these principles, you can write cleaner, efficient, and robust Python code.

In [ ]:
# Get Llama 3.2 to answer